In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd

In [32]:
df_raw = (
    pd.read_csv('network_df.csv')
    .assign(owned_asset_count = lambda x: x.owned_asset_count.astype(float))
)
df_raw.info()
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188512 entries, 0 to 188511
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dao_id             188512 non-null  object 
 1   member_address     188512 non-null  object 
 2   slug               188512 non-null  object 
 3   owned_asset_count  188512 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.8+ MB


dao_id  \
0  ec242c8a-4667-46c6-8c51-0df6f0970970   
1  f12e8672-59e5-425e-bfd9-42f4078a3ff1   
2  f12e8672-59e5-425e-bfd9-42f4078a3ff1   
3  f12e8672-59e5-425e-bfd9-42f4078a3ff1   
4  f12e8672-59e5-425e-bfd9-42f4078a3ff1   

                               member_address  \
0  0x9533158bb9b0abb178a9dc7f6f0168e84126688f   
1  0x097c39e5e576a8706404cd0d81e05b522f5bcaff   
2  0x097c39e5e576a8706404cd0d81e05b522f5bcaff   
3  0x097c39e5e576a8706404cd0d81e05b522f5bcaff   
4  0x097c39e5e576a8706404cd0d81e05b522f5bcaff   

                                    slug  owned_asset_count  
0  movement-daos-ascended-ape-collection                1.0  
1                           eth-merge-v3                1.0  
2                          grand-leisure                1.0  
3               re-based-ghouls-by-xwave                1.0  
4                           based-ghouls                6.0

In [38]:
slug = (
    df_raw
    .groupby('slug')
    .owned_asset_count.sum()
    .sort_values(ascending = False)
    .to_frame()
)
slug

owned_asset_count
slug                                               
unidentified-contract-vzooewvccc       1.579209e+76
unidentified-contract-cdgfjphfbh       2.278085e+38
unidentified-contract-kgoq4pkam0       9.709694e+23
unidentified-contract-ytlksgcesz       7.281004e+23
unidentified-contract-kj63urtefs       5.053004e+22
...                                             ...
friedguyz                              1.000000e+00
sudoangels-limited                     1.000000e+00
friendly-fire-v2                       1.000000e+00
sudo-rm-rf                             1.000000e+00
femversenft                            1.000000e+00

[28446 rows x 1 columns]

In [52]:
top_N = 20
slug_top = slug.head(top_N).index.tolist()
df = df_raw.loc[lambda x: x.slug.isin(slug_top)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 165 to 187152
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dao_id             68 non-null     object 
 1   member_address     68 non-null     object 
 2   slug               68 non-null     object 
 3   owned_asset_count  68 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.7+ KB


# Data for network visualization
- Source
- Target
- weight

In [53]:
from itertools import permutations
from tqdm import tqdm
import networkx as nx
from pyvis.network import Network
import io, pickle

In [54]:
dir_path = 'vis'

# DAO network

In [67]:
network_dao = {}
for grp, df_grp in tqdm(df.groupby('slug')):
    for p in permutations(sorted(df_grp.dao_id.unique()), 2):
        if p not in network_dao:
            network_dao[p] = 0
        network_dao[p] += 1

100%|██████████| 20/20 [00:00<00:00, 2155.18it/s]


In [68]:
df_network_dao = pd.DataFrame([{'Source': source, 'Target': target, 'weight': weight} for (source, target), weight in network_dao.items()])
df_network_dao.to_csv(f'{dir_path}/vis_network_dao.csv', index = False)

In [69]:
df_network_dao.info()
df_network_dao.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Source  710 non-null    object
 1   Target  710 non-null    object
 2   weight  710 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.8+ KB


Source                                Target  \
0  0519572c-483c-4d6e-ac6d-28eda3ea1aff  11eb10d7-6493-4cfc-8255-cb0f2338fa67   
1  0519572c-483c-4d6e-ac6d-28eda3ea1aff  219ab5ef-9345-448b-bbf2-786a33e4edd8   
2  0519572c-483c-4d6e-ac6d-28eda3ea1aff  50633191-27ac-40e3-9f52-da100c196c90   
3  0519572c-483c-4d6e-ac6d-28eda3ea1aff  5c76f492-c718-4c2e-b633-c9dbac1cc331   
4  0519572c-483c-4d6e-ac6d-28eda3ea1aff  67149dec-7a5d-41f4-8e68-f75446ce47a0   

   weight  
0       1  
1       1  
2       1  
3       1  
4       1

In [70]:
G_dao = nx.from_pandas_edgelist(
    df_network_dao, 
    source='Source', 
    target='Target',
    edge_attr='weight')

with io.open(f'{dir_path}/gragh_dao.nx', mode = 'wb') as f:
    pickle.dump(G_dao, f)

In [71]:
net_dao = Network(notebook = True)
net_dao.from_nx(G_dao)
net_dao.show(f'{dir_path}/pyvis_nx_dao.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Voter network

In [62]:
network_voters = {}
for grp, df_grp in tqdm(df.groupby('slug')):
    for p in permutations(sorted(df_grp.member_address.unique()), 2):
        if p not in network_voters:
            network_voters[p] = 0
        network_voters[p] += 1

100%|██████████| 20/20 [00:00<00:00, 1253.38it/s]


In [63]:
df_network_voter = pd.DataFrame([{'Source': source, 'Target': target, 'weight': weight} for (source, target), weight in network_voters.items()])
df_network_voter.to_csv(f'{dir_path}/vis_network_voter.csv', index = False)

In [64]:
df_network_voter.info()
df_network_voter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Source  320 non-null    object
 1   Target  320 non-null    object
 2   weight  320 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.6+ KB


Source  \
0  0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01   
1  0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01   
2  0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01   
3  0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01   
4  0x0998160bdf3ff6d86a4e9d5c31e0efc3ca7e7d01   

                                       Target  weight  
0  0x14bdfda5b5b829f14332a52c15129386284ce36a       1  
1  0x194f0fe6d4dfbf198868b1b507cd74122fbfebcc       1  
2  0x1ca48a32482ef29708e95f248e8a613f05782ed3       1  
3  0x24e6d835f23b9716af984f6604625c6b074015cd       1  
4  0x4194ce73ac3fbbece8ffa878c2b5a8c90333e724       1

In [65]:
G_voter = nx.from_pandas_edgelist(
    df_network_voter, 
    source='Source', 
    target='Target',
    edge_attr='weight')

with io.open(f'{dir_path}/gragh_voter.nx', mode = 'wb') as f:
    pickle.dump(G_voter, f)

In [66]:
net_voter = Network(notebook = True)
net_voter.from_nx(G_voter)
net_voter.show(f'{dir_path}/pyvis_nx_voter.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
